<a href="https://colab.research.google.com/github/lancal/Tarea02Spark/blob/master/Tarea02Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!ls

sample_data  spark-2.3.3-bin-hadoop2.7	spark-2.3.3-bin-hadoop2.7.tgz


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [4]:
!java --version


openjdk 11.0.3 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
sqlSession = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

**Nota: Se recomienda usar el archivo master.csv que se encuentra en el archivo .zip en vez del original descargado de la pagina de kaggle.com, ya que por problemas de numeros digitales en norma americana  ,por ejemplo ("100,000"). En una de las columnas no fue posible eliminar de manera satisfactoria con pyspark, ya que tambien hubo complicaciones con las comas (,) en leer las lineas del archivo, por lo que se elimino previamente al ser cargado en colab con el metodo para subir archivo.

In [7]:
from google.colab import files

#drive.mount('drive');

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving master.csv to master.csv
User uploaded file "master.csv" with length 2228051 bytes


# **Import libraries** 

In [0]:
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


import math
import pandas as pd
import matplotlib.pylab as plt

# **Load and prepare data**

Se crea RDD  y se almacena en cache, posteriormente se imprimen las 5 primeras lineas sin prepocesamiento ni limpieza.

In [9]:
suicidesRDD = sc.textFile("master.csv")
suicidesRDD.cache()
#suicidesRDD = suicidesRDD.flatMap(lambda t:t.split(","))
suicidesRDD.take(5)
#print (top5)

['country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_per_capita ($),generation',
 'Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,,796,Generation X',
 'Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,,796,Silent',
 'Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,,796,Generation X',
 'Albania,1987,male,75+ years,1,21800,4.59,Albania1987,,796,G.I. Generation']

Se Elimina el Encabezado:

In [10]:
header = suicidesRDD.first()
datalines = suicidesRDD.filter(lambda x : x != header)
#suicidesRDD.drop(' gdp_for_year ($) ').collect()
#print(suicidesRDD.collect())
#datalines = suicidesRDD.drop('gdp_for_year ($)').collect()
#datalines[9].dataType = IntegerType()
#suicidesRDD['gdp_for_year ($)'].dataType = IntegerType()


datalines.count()
datalines.take(5)

['Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,,796,Generation X',
 'Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,,796,Silent',
 'Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,,796,Generation X',
 'Albania,1987,male,75+ years,1,21800,4.59,Albania1987,,796,G.I. Generation',
 'Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,,796,Boomers']

Se define funcion para limpieza de cada linea del RDD,

En esta parte se uso el siguiente critero para clasificar y reemplazar algunas columnas tales como:

AGE:

"5-14 years" -> 1

"15-24 years"  -> 2

 "25-34 years" -> 3
 
"35-54 years" -> 4

"55-74 years" -> 5

 "75+ years" -> 6

GENERATION: 

"Boomers" -> 1

"G.I. Generation" -> 2

"Generation X" -> 3

"Generation Z" -> 4

"Millenials" -> 5

 "Silent" -> 6
 
 SEX:
 
"male" -> 1

"female" -> 2

In [0]:
def cleanseRDD(inputStr) :
  
    #if isinstance(inputStr, int) :
    #    return inputStr
      
    #Eliminar las comillas
    #repStr = inputStr.replace('\"','-')
      
    attList=inputStr.split(",")
    
    if attList[2] == "male" :
         attList[2]="1"
    elif attList[2] == "female" :
         attList[2]="2"
        
    if attList[3] == "5-14 years":
         attList[3] = "1"
    elif attList[3] == "15-24 years" :
         attList[3]="2"
    elif attList[3] == "25-34 years" :
         attList[3]="3"
    elif attList[3] == "35-54 years" :
         attList[3]="4"
    elif attList[3] == "55-74 years" :
         attList[3]="5"
    elif attList[3] == "75+ years" :
         attList[3]="6"
        
    if attList[8] == "":
         attList[8] = "0"
        
    if attList[10] == "Boomers":
         attList[10] = "1"
    elif attList[10] == "G.I. Generation" :
         attList[10]="2"
    elif attList[10] == "Generation X" :
         attList[10]="3"
    elif attList[10] == "Generation Z" :
         attList[10]="4"
    elif attList[10] == "Millenials" :
         attList[10]="5"
    elif attList[10] == "Silent" :
         attList[10]="6"
        
    #Create a row with cleaned up and converted data
    
    values = Row(  COUNTRY = attList[0],\
                   YEAR = float(attList[1]),\
                   SEX = float(attList[2]),\
                   AGE = float(attList[3]),\
                   SUICIDES_NO = float(attList[4]),\
                   POPULATION = float(attList[5]),\
                   SUICIDES_HUNDREDK_POP = float(attList[6]),\
                   COUNTRY_YEAR = attList[7], \
                   HDI_FOR_YEAR = float(attList[8]),\
                   GDP_PER_YEAR = float(attList[9]),\
                   GENERATION = float(attList[10]) ) 
    
        
    #attList[9].dataType = IntegerType()
  
    #Convert Drive to uppercase
    #attList[5] = attList[5].upper()
    #return ",".join(attList)
  
    return values

In [59]:
suicidesData = datalines.map(cleanseRDD)
suicidesData.cache()
suicidesData.collect()
suicidesData.take(50)
#for line in suicidesData.take(500):
#  print(line)

[Row(AGE=2.0, COUNTRY='Albania', COUNTRY_YEAR='Albania1987', GDP_PER_YEAR=796.0, GENERATION=3.0, HDI_FOR_YEAR=0.0, POPULATION=312900.0, SEX=1.0, SUICIDES_HUNDREDK_POP=6.71, SUICIDES_NO=21.0, YEAR=1987.0),
 Row(AGE=4.0, COUNTRY='Albania', COUNTRY_YEAR='Albania1987', GDP_PER_YEAR=796.0, GENERATION=6.0, HDI_FOR_YEAR=0.0, POPULATION=308000.0, SEX=1.0, SUICIDES_HUNDREDK_POP=5.19, SUICIDES_NO=16.0, YEAR=1987.0),
 Row(AGE=2.0, COUNTRY='Albania', COUNTRY_YEAR='Albania1987', GDP_PER_YEAR=796.0, GENERATION=3.0, HDI_FOR_YEAR=0.0, POPULATION=289700.0, SEX=2.0, SUICIDES_HUNDREDK_POP=4.83, SUICIDES_NO=14.0, YEAR=1987.0),
 Row(AGE=6.0, COUNTRY='Albania', COUNTRY_YEAR='Albania1987', GDP_PER_YEAR=796.0, GENERATION=2.0, HDI_FOR_YEAR=0.0, POPULATION=21800.0, SEX=1.0, SUICIDES_HUNDREDK_POP=4.59, SUICIDES_NO=1.0, YEAR=1987.0),
 Row(AGE=3.0, COUNTRY='Albania', COUNTRY_YEAR='Albania1987', GDP_PER_YEAR=796.0, GENERATION=1.0, HDI_FOR_YEAR=0.0, POPULATION=274300.0, SEX=1.0, SUICIDES_HUNDREDK_POP=3.28, SUICIDES_

In [225]:
suicidesDf = sqlSession.createDataFrame(suicidesData)
suicidesDf.show()

+---+-------+------------+------------+----------+------------+----------+---+---------------------+-----------+------+
|AGE|COUNTRY|COUNTRY_YEAR|GDP_PER_YEAR|GENERATION|HDI_FOR_YEAR|POPULATION|SEX|SUICIDES_HUNDREDK_POP|SUICIDES_NO|  YEAR|
+---+-------+------------+------------+----------+------------+----------+---+---------------------+-----------+------+
|2.0|Albania| Albania1987|       796.0|       3.0|         0.0|  312900.0|1.0|                 6.71|       21.0|1987.0|
|4.0|Albania| Albania1987|       796.0|       6.0|         0.0|  308000.0|1.0|                 5.19|       16.0|1987.0|
|2.0|Albania| Albania1987|       796.0|       3.0|         0.0|  289700.0|2.0|                 4.83|       14.0|1987.0|
|6.0|Albania| Albania1987|       796.0|       2.0|         0.0|   21800.0|1.0|                 4.59|        1.0|1987.0|
|3.0|Albania| Albania1987|       796.0|       1.0|         0.0|  274300.0|1.0|                 3.28|        9.0|1987.0|
|6.0|Albania| Albania1987|       796.0| 

# **Data Analytics**

In [227]:
suicidesDf.select("SUICIDES_NO","POPULATION","SUICIDES_HUNDREDK_POP","SEX").describe().show()

+-------+------------------+------------------+---------------------+------------------+
|summary|       SUICIDES_NO|        POPULATION|SUICIDES_HUNDREDK_POP|               SEX|
+-------+------------------+------------------+---------------------+------------------+
|  count|             27820|             27820|                27820|             27820|
|   mean|242.57440690150972|1844793.6173975556|   12.816097411933846|               1.5|
| stddev| 902.0479168336407|3911779.4417563635|   18.961511014503138|0.5000089865830324|
|    min|               0.0|             278.0|                  0.0|               1.0|
|    max|           22338.0|       4.3805214E7|               224.97|               2.0|
+-------+------------------+------------------+---------------------+------------------+



In [62]:
for i in suicidesDf.columns:
  if not isinstance(suicidesDf.select(i).take(1)[0][0], str):
    print( "Correlation to SUICIDES_NO for ", i, suicidesDf.stat.corr('SUICIDES_NO',i))

Correlation to SUICIDES_NO for  AGE 0.07989235705323104
Correlation to SUICIDES_NO for  GDP_PER_YEAR 0.06132974884024581
Correlation to SUICIDES_NO for  GENERATION -0.0706237053792957
Correlation to SUICIDES_NO for  HDI_FOR_YEAR -0.017307725751743584
Correlation to SUICIDES_NO for  POPULATION 0.616162267521927
Correlation to SUICIDES_NO for  SEX -0.1446291743414448
Correlation to SUICIDES_NO for  SUICIDES_HUNDREDK_POP 0.3066044512677835
Correlation to SUICIDES_NO for  SUICIDES_NO 1.0
Correlation to SUICIDES_NO for  YEAR -0.00454595750741003


# **Prepare data**

Transform to a Data Frame for input to Machine Learing

Drop columns that are not required (low correlation)

In [0]:
def transformToLabeledPoint(row) :
    lp = ( row["SUICIDES_NO"], Vectors.dense([row["POPULATION"], \
                        #                     
                        row["SUICIDES_HUNDREDK_POP"], \
                        #row["POPULATION"],
                        row["SEX"], \
                        row["GENERATION"]
                        #row["YEAR"]
                                             ]))
    return lp

In [194]:
suicidesLP = suicidesData.map(transformToLabeledPoint)
suicidesDf = sqlSession.createDataFrame(suicidesLP,["label", "features"])
suicidesDf.select("label","features").show(20)

+-----+-------------------+
|label|           features|
+-----+-------------------+
| 21.0|[312900.0,6.71,1.0]|
| 16.0|[308000.0,5.19,1.0]|
| 14.0|[289700.0,4.83,2.0]|
|  1.0| [21800.0,4.59,1.0]|
|  9.0|[274300.0,3.28,1.0]|
|  1.0| [35600.0,2.81,2.0]|
|  6.0|[278800.0,2.15,2.0]|
|  4.0|[257200.0,1.56,2.0]|
|  1.0|[137500.0,0.73,1.0]|
|  0.0| [311000.0,0.0,2.0]|
|  0.0| [144600.0,0.0,2.0]|
|  0.0| [338200.0,0.0,1.0]|
|  2.0| [36400.0,5.49,2.0]|
| 17.0|[319200.0,5.33,1.0]|
|  1.0| [22300.0,4.48,1.0]|
| 14.0|[314100.0,4.46,1.0]|
|  4.0|[140200.0,2.85,1.0]|
|  8.0|[295600.0,2.71,2.0]|
|  3.0|[147500.0,2.03,2.0]|
|  5.0|[262400.0,1.91,2.0]|
+-----+-------------------+
only showing top 20 rows



# **Linear Regression**

El modelo escogido para hacer prediccion es el modelo de regresion lineal

Split into training and testing data

In [212]:
#Se genera un random Split del total de datos dejgando un 80% para training data
# y un 20% para testing data
(trainingData, testData) = suicidesDf.randomSplit([0.8, 0.2])
#Se imprimen la cantidad de training Data
print(trainingData.count())
#Se imprimen la cantidad de Test Data
print(testData.count())

22289
5531


Build the model on training data

In [0]:
lr = LinearRegression(maxIter=500)
lrModel = lr.fit(trainingData)

Print the metrics

In [221]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [0.00014344259770679774,13.961652484202293,-69.7335386984349]
Intercept: -95.55300275199573


Predict on the test data

In [222]:
predictions = lrModel.transform(testData)
predictions.select("prediction","label","features").show(1000)

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
| -165.2182627739222|  0.0|     [476.0,0.0,1.0]|
|-234.94520311286257|  0.0|     [522.0,0.0,2.0]|
|-165.20936933286436|  0.0|     [538.0,0.0,1.0]|
|-165.20004556401344|  0.0|     [603.0,0.0,1.0]|
|-165.15744311249452|  0.0|     [900.0,0.0,1.0]|
|-165.15328327716102|  0.0|     [929.0,0.0,1.0]|
|-234.86501870074449|  0.0|    [1081.0,0.0,2.0]|
|-165.12961524853938|  0.0|    [1094.0,0.0,1.0]|
| -165.1273201669761|  0.0|    [1110.0,0.0,1.0]|
|-165.10566033472236|  0.0|    [1261.0,0.0,1.0]|
|-165.10164394198657|  0.0|    [1289.0,0.0,1.0]|
| -165.0982013196416|  0.0|    [1313.0,0.0,1.0]|
|-165.09748410665307|  0.0|    [1318.0,0.0,1.0]|
|-165.09217673053791|  0.0|    [1355.0,0.0,1.0]|
| -165.0875865674113|  0.0|    [1387.0,0.0,1.0]|
|-165.07984066713516|  0.0|    [1441.0,0.0,1.0]|
|-234.81194493959296|  0.0|    [1451.0,0.0,2.0]|
| -234.8067810060755

Find R2 for Linear Regression

In [223]:
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
evaluator.evaluate(predictions)

0.46027837626640244